## Exercise 2.53
**What would the interpreter print in response
to evaluating each of the following expressions?**

**`(list 'a 'b 'c)`**

**`(list (list 'george'))`**

**`(cdr '((x1 x2) (y1 y2)))`**

**`(cadr '((x1 x2) (y1 y2)))`**

**`(pair? (car '(a short list)))`**

**`(memq 'red '((red shoes) (blue socks)))`**

**`(memq 'red '(red shoes blue socks))`**

The interpreter would print, respectively:

`(a b c)`

`((george))`

`((y1 y2))`

`(y1 y2)`

`#f`

`#f`

`(red shoes blue socks)`

## Exercise 2.54
**Two lists are said to be `equal?` if they contain equal elements arranged in the
same order. For example,**
```
(equal? '(this is a list) '(this is a list))
```
**is true, but**
```
(equal? '(this is a list) '(this (is a) list))
```
**is false. To be more precise, we can define `equal?` recursively in terms of
the basic `eq?` equality of symbols by saying that `a` and `b` are `equal?` if
they are both symbols and the symbols are `eq?`, or if they are both lists
such that `(car a)` is `equal?` to `(car b)` and `(cdr a)` is `equal?` to `(cdr b)`.
Using this idea, implement `equal?` as a procedure.**

In [1]:
(define (equal? first second)
  (cond ((not (or (pair? first) (pair? second))) (eq? first second))
        ((and (pair? first) (pair? second)) (and (equal? (car first) (car second))
                                                  (equal? (cdr first) (cdr second))))
        (else #f)))

(equal? '(this is a list) '(this is a list))

#t

In [2]:
(equal? '(this is a list) '(this (is a) list))

#f

## Exercise 2.55
**Eva Lu Ator types to the interpreter the expression**
```
(car ''abracadabra)
```
**To her surprise, the interpreter prints bakck `quote`. Explain.**

It seems that the sub-expression `''abracadabra` is being parsed as a quoted list of the two symbols that follow the first quote, i.e. a list comprising a quote symbol and the symbol 'abracadabra'. So, the `car` of the list is a `quote` and the `cdr` is `'(abracadabra)`.

## Exercise 2.56
**Show how to extend the basic differentiator to handle more kinds of
expressions. For instance, implement the differentiation rule**
$$
\frac{d(u^n)}{dx} = nu^{n-1}\frac{du}{dx}
$$
**by adding a new clause to the `deriv` program and defining
appropriate procedures `exponentiation?`, `base`, `exponent`,
and `make-exponentiation`. (You may use the symbol `**`
to denote exponentiation.) Build in the rules that anything
raised to the power 0 is 1 and anything raised to the power
1 is the thing itself.**


First, we need to bring over the definitions from the text to use later.

In [3]:
(define (variable? x) (symbol? x))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (make-sum a1 a2) (list '+ a1 a2))

(define (make-product m1 m2) (list '* m1 m2))

(define (sum? x) (and (pair? x) (eq? (car x) '+)))

(define (addend s) (cadr s))

(define (augend s) (caddr s))

(define (product? x) (and (pair? x) (eq? (car x) '*)))

(define (multiplier p) (cadr p))

(define (multiplicand p) (caddr p))
  
(define (make-sum a1 a2) 
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list '+ a1 a2))))

(define (=number? exp num) (and (number? exp) (= exp num)))

(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2))))

Then we will need constructors and selectors for exponentiation:

In [4]:
(define (make-exponentiation base exponent)
  (cond ((=number? base 0) 0)
        ((=number? base 1) 1)
        ((=number? exponent 0) 1)
        ((=number? exponent 1) base)
        ((and (number? base) (number? exponent)) (exp base exponent))
        (else (list '** base exponent))))

(define (base expr) (cadr expr))
(define (exponent expr) (caddr expr))
(define (exponentiation? expr) (eq? '** (car expr)))

Now we can update the definition of `deriv` to include a rule for exponents:

In [5]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        ((sum? exp) (make-sum (deriv (addend exp) var)
                              (deriv (augend exp) var)))
        ((product? exp)
         (make-sum
           (make-product (multiplier exp)
                         (deriv (multiplicand exp) var))
           (make-product (deriv (multiplier exp) var)
                         (multiplicand exp))))
        ((exponentiation? exp)
         (make-product (make-product (exponent exp)
                                     (make-exponentiation (base exp) (make-sum (exponent exp) -1)))
                       (deriv (base exp) var)))
        (else
          (error "unknown expression type: DERIV" exp))))

It can now be used to differentiate exponentiated expressions, like:

In [6]:
(deriv '(** x 4) 'x)

'(* 4 (** x 3))

## Exericse 2.57
**Extend the differentiation program to handle sums and products of arbitrary numbers of (two or more)
terms. Then the last example above could be expressed as**
```
(deriv '(* x y (+ x 3)) 'x)
```
**Try to do this by changing only the representation for sums
and products, without changing the `deriv` procedure at all.
For example, the `addend` of a sum would be the first term,
and the `augend would be the sum of the rest of the terms.**


In [7]:
; Takes a list of the addends as the sole argument
(define (make-sum-from-list a)
  (define numbers-sum (foldl + 0 (filter number? a)))
  (define non-numbers (filter (lambda (x) (not (number? x))) a))
  (cond ((= (length a) 1) (car a))
        ((= (length non-numbers) 0) numbers-sum)
        ((= numbers-sum 0) (if (= (length non-numbers) 1)
                               (car non-numbers)
                               (cons '+ non-numbers)))
        (else (append (list '+ numbers-sum) non-numbers))))

; Takes each addend as a separate argument
(define (make-sum . a)
  (make-sum-from-list a))

(define (addend sum) (cadr sum))
(define (augend sum) (make-sum-from-list (cddr sum)))

; Takes a list of the multipliers as the sole argument
(define (make-product-from-list p)
  (define numbers-product (foldl * 1 (filter number? p)))
  (define non-numbers (filter (lambda (x) (not (number? x))) p))
  (cond ((= 0 numbers-product) 0)
        ((= 0 (length non-numbers)) numbers-product)
        ((= 1 (length p)) (car p))
        ((= 1 numbers-product) (if (= 1 (length non-numbers))
                                   (car non-numbers)
                                   (cons '* non-numbers)))
        (else (append (list '* numbers-product) non-numbers))))

; Takes each multiplier as a separate argument
(define (make-product . p)
  (make-product-from-list p))

(define (multiplier p) (cadr p))
(define (multiplicand p) (make-product-from-list (cddr p)))

Now we can compute derivatives of expressions like the one given in the question:

In [8]:
(deriv '(* x y (+ x 3)) 'x)

'(+ (* x y) (* y (+ x 3)))

## Exercise 2.58
**Suppose we want to modify the differentiation program so that it works with ordinary mathematical
notation, in which `+` and `*` are infix rather than prefix operators. Since the
differentiation program is defined in terms of abstract data, we can modify it
to work with different representations of expressions solely by changing the predicates,
selectors, and constructors that define the representation of the algebraic
expressions on which the differentiator is to
operate.**

a. **Show how to do this in order to differentiate algebraic expressions presented
   in infix form, such as `(x + (3 * (x + (y + 2))))`. To simplify the task, assume that
   `+` and `*` always take two arguments and that expressions are fully parenthesized.**

b. **The problem becomes substantially harder if we allow
   standard algebraic notation, such as `(x + 3 * (x + y + 2))`, which drops
   unnecessary parentheses and assumes that multiplication is done before addition.
   Can you design appropriate predicates, selectors, and
   constructors for this notation such that our derivative
   program still works?**
                        

a. This case is quite easy, in fact it is simpler than Exercise 2.57 because each operator here will have only two operands. We just need to make a few changes:

In [9]:
; First, to sum? and product? so that we check the second element in the list to find the operator
(define (sum? x) (and (pair? x) (eq? (cadr x) '+)))
(define (product? x) (and (pair? x) (eq? (cadr x) '*)))

; Then, we alter make-sum and make-product
(define (make-sum a1 a2) (list a1 '+ a2))
(define (make-product m1 m2) (list m1 '* m2))

; And update the accessors so they access the correct element
(define (addend s) (car s))
(define (augend s) (caddr s))
(define (multiplier p) (car p))
(define (multiplicand p) (caddr p))

; With all that done, we can now differentiate infix expressions:
(deriv '(x + (3 * (x + (y + 2)))) 'x)

'(1 + ((3 * (1 + (0 + 0))) + (0 * (x + (y + 2)))))

The above expression is not in the simplest form, but evaluates to 4, which is the correct answer. With some modifications to the constructors, we can enable simplification as well:

In [10]:
; These are the same definitions as in Exercise 2.56, but with the order of the operator altered.
(define (make-sum a1 a2) 
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2))
         (+ a1 a2))
        (else (list a1 '+ a2))))

(define (=number? exp num) (and (number? exp) (= exp num)))

(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2)
        ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list m1 '* m2))))

(deriv '(x + (3 * (x + (y + 2)))) 'x)

4

b. This case is much more difficult. It akin to writing a parser for algebraic expressions, but with the benefit that the lexing has been already done and the parsing for parentheses has already been done.

In [11]:
(define (push l i)
  (append l (list i)))

(define (parse expr)
  (define (parse-mul-iter rem partial)
    (cond ((= 1 (length rem)) (make-product-from-list (push partial (parse (car rem)))))
          ((eq? '* (cadr rem)) (parse-mul-iter (cddr rem) (push partial (parse (car rem)))))
          ((eq? '+ (cadr rem)) (parse-sum-iter (cddr rem)
                                               (list (make-product-from-list (push partial (car rem))))))))
  (define (parse-sum-iter rem partial)
    (cond ((= 1 (length rem)) (make-sum-from-list (push partial (parse (car rem)))))
          ((eq? '+ (cadr rem)) (parse-sum-iter (cddr rem) (push partial (parse (car rem)))))
          ((eq? '* (cadr rem)) (make-sum-from-list (push partial (parse-mul-iter rem '()))))))
  (cond ((number? expr) expr)
        ((symbol? expr) expr)
        (else (parse-sum-iter expr '()))))

; We also need to update definitions of constructors and accessors to be compatible with the list-based format
; These are just copied from Exercise 2.57
(define (make-sum . a)
  (make-sum-from-list a))
(define (addend sum) (cadr sum))
(define (augend sum) (make-sum-from-list (cddr sum)))

(define (make-product . p)
  (make-product-from-list p))
(define (multiplier p) (cadr p))
(define (multiplicand p) (make-product-from-list (cddr p)))

(define (sum? x) (and (pair? x) (eq? (car x) '+)))
(define (product? x) (and (pair? x) (eq? (car x) '*)))



With `parse` defined to bring standard algebraic notation into a format our procedures understand, we can now run `deriv` on these expressions:

In [12]:
(deriv (parse '(x + 3 * (x + y + 2))) 'x)

4

## Exercise 2.59
**Implement the `union-set` operation for the unordered-list representation of sets.**

In [13]:
; First we bring definitions from the books for later use
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((equal? x (car set)) #t)
        (else (element-of-set? x (cdr set)))))

(define (adjoin-set x set)
  (if (element-of-set? x set)
      set
      (cons x set)))

(define (intersection-set set1 set2)
  (cond ((or (null? set1) (null? set2)) '())
        ((element-of-set? (car set1) set2)
         (cons (car set1) (intersection-set (cdr set1) set2)))
        (else (intersection-set (cdr set1) set2))))

In [14]:
(define (union-set set1 set2)
  (cond ((null? set1) set2)
        ((element-of-set? (car set1) set2) (union-set (cdr set1) set2))
        (else (union-set (cdr set1) (adjoin-set (car set1) set2)))))

(union-set (list 1 2 3) (list 2 3 4))

'(1 2 3 4)

## Exercise 2.60
**We specified that a set would be represented
as a list with no duplicates. Now suppose we allow duplicates. For instance, the set $\{1, 2, 3\}$ could be represented as
the list `(2 3 2 1 3 2 2)`. Design procedures `element-of-set?`, `adjoin-set`, `union-set`, and `intersection-set`
that operate on this representation. How does the efficiency
of each compare with the corresponding procedure for the 
non-duplicate representation? Are there applications for which
you would use this representation in preference to the non-duplicate one?**

In [15]:
; We can leave element-of-set? unchanged, as allowing duplicates
; does not make a difference in its operation.

; For adjoin-set, we can cons the item to the set without checking if it is already there
(define (adjoin-set x set)
  (cons x set))

; Likewise for union-set, we can append the two lists without caring for if any common elements exist\
(define (union-set set1 set2)
  (append set1 set2))

; We can also leave the implementation of intersection-set unchanged, as it should work ok
; for lists with repeated elements.

The efficiency of `element-of-set?` and `intersection-set` does not change in the new representation with respect to the number of items in the lists we are using to represent the sets, as we are not changing these functions. However, the number of items in the list will no longer be necessarily equal to the number of items in the represented set. The initial set construction and applications of `adjoin-set` and `union-set` could have increased the number of items in the list by a lot even when the set has only a few elements. However, making the assumption that this effect isn't significant, the efficiency of `element-of-set?` and `intersection-set` remains the same, that is $\Theta(n)$ and $\Theta(n^2)$ respectively.

However, `adjoin-set` and `union-set` are now fast due to the simplifications we made. `adjoin-set` uses a single `cons` operation, which makes it a $\Theta(1)$ function. `union-set` uses one `append` operation, so it is a $\Theta(n)$ function, where $n$ is the number of items in the first set. This is because `append` has to walk the first list in its entirety to find the end where it can attach the second list.

I would use this representation of sets in applications where storage is aplenty and a lot of `adjoin-set` operations and `union-set` operations are needed compared to the other two operations. 

## Exercise 2.61
**Give an implementation of `adjoin-set` using the ordered representation. By analogy with
`element-of-set?` show how to take advantage of the ordering to
produce a procedure that requires on the average about half
as many steps as with the unordered representation.**

In [16]:
; Again, bringing over some definitions from the book for this representation
(define (element-of-set? x set)
  (cond ((null? set) false)
        ((= x (car set)) true)
        ((< x (car set)) false)
        (else (element-of-set? x (cdr set)))))

(define (intersection-set set1 set2)
  (if (or (null? set1) (null? set2))
      '()
      (let ((x1 (car set1)) (x2 (car set2)))
        (cond ((= x1 x2)
               (cons x1 (intersection-set (cdr set1)
                                          (cdr set2))))
              ((< x1 x2)
               (intersection-set (cdr set1) set2))
              ((< x2 x1)
               (intersection-set set1 (cdr set2)))))))

(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((= x (car set)) set)
        ((< x (car set)) (cons x set))
        (else (cons (car set) (adjoin-set x (cdr set))))))

(adjoin-set 5 (list 1 2 4 8 16))

'(1 2 4 5 8 16)

The above implementation of `adjoin-set` takes advantage of the ordering to reduce the average running time to about half compared to the previous representation. This is because it, starting from the beginning, looks for the element that is greater than `x`, and when it finds one, the search ends there. (It does take additional steps to `cons` up the elements after that point has been found, so I am not *certain* the running time will be about half.)

## Exercise 2.62
**Give a $\Theta(n)$ implementation of `union-set` for sets represented as ordered lists.**

In [17]:
(define (union-set set1 set2)
  (cond ((null? set1) set2)
        ((null? set2) set1)
        (else (let ((x1 (car set1)) (x2 (car set2)))
                (cond ((= x1 x2) (cons x1 (union-set (cdr set1) (cdr set2))))
                      ((< x1 x2) (cons x1 (union-set (cdr set1) set2)))
                      ((> x1 x2) (cons x2 (union-set set1 (cdr set2)))))))))

(union-set (list 101 102 103) (list 99 100))

'(99 100 101 102 103)

The above implementation of `union-set` runs in $\Theta(n)$ time because in each step, it makes a recursive call, reducing the size of one or both of the input sets by one element. In the worst case, it will take $2n$ recursive steps, where $n$ is the number of elements in each set. Since $2n$ is still $\Theta(n)$, this implementation of `union-set` is $\Theta(n)$.

## Exercise 2.63
**Each of the following two procedures converts a binary tree to a list.**
```
(define (tree->list-1 tree)
  (if (null? tree)
    '()
    (append (tree->list-1 (left-branch tree))
            (cons (entry tree)
                  (tree->list-1
                    (right-branch tree))))))
(define (tree->list-2 tree)
  (define (copy-to-list tree result-list)
    (if (null? tree)
        result-list
        (copy-to-list (left-branch tree)
                      (cons (entry tree)
                            (copy-to-list
                              (right-branch tree)
                              result-list)))))
  (copy-to-list tree '()))
```
**a. Do the two procedures produce the same result for
every tree? If not, how do the results differ? What lists do the two procedures produce for the trees in Figure 2.16?**

**b. Do the two procedures have the same order of growth
in the number of steps required to convert a balanced
tree with $n$ elements to a list? If not, which one grows
more slowly?**

a. Yes, the two procedures produce the same result for every tree. This is becasue the they follow the same pattern of converting the right subtree into a list, 'cons'ing it after the root, and 'cons'ing the result after the result of converting the left subtree into a list. This produces a list that corresponds to what is called an 'inorder' walk of the tree.

b. The two procedures have the same order of growth in the number of steps. This is because at each step, both of them apply the same operation recursively to the left subtree and the right subtree. This leads to a running time that is on the order of $\Theta(n)$ where $n$ is the number of items in the tree. However, if we take into account the fact that `append`-ing is not a one-step operation, this changes. Since `append` can take as many steps as the length of the first argument, the running time of the first function grows as $nlog_2(n)$.

## Exercise 2.64
**The following procedure `list->tree` converts an ordered list to a balanced
binary tree. The helper procedure `partial-tree` takes as arguments an integer
$n$ and list of at least $n$ elements and constructs a balanced
tree containing the first $n$ elements of the list. The result
returned by `partial-tree` is a pair (formed with `cons`) whose
`car` is the constructed tree and whose `cdr` is the list of ele-
ments not included in the tree.**

```
(define (list->tree elements)
  (car (partial-tree elements (length elements))))
(define (partial-tree elts n)
  (if (= n 0)
    (cons '() elts)
    (let ((left-size (quotient (- n 1) 2)))
      (let ((left-result
              (partial-tree elts left-size)))
        (let ((left-tree (car left-result))
              (non-left-elts (cdr left-result))
              (right-size (- n (+ left-size 1))))
          (let ((this-entry (car non-left-elts))
                (right-result
                  (partial-tree
                    (cdr non-left-elts)
                    right-size)))
            (let ((right-tree (car right-result))
                  (remaining-elts
                    (cdr right-result)))
              (cons (make-tree this-entry
                               left-tree
                               right-tree)
                    remaining-elts))))))))
```

**a. Write a short paragraph explaining as clearly as you can how `partial-tree`
works. Draw the tree produced
by `list->tree` for the list `(1 3 5 7 9 11)`.**

**b. What is the order of growth in the number of steps required by `list->tree`
to convert a list of $n$ elements?**

a. `partial-tree` works in a divide-and-conquer fashion by splitting the tree in (roughly) half at every step. First it take the left half of the list, and recursively applies itself to convert it into a tree. Then, since the function returns the 'unused' elements if the argument $n$ is less than the size of the list of elements it's given, it uses those leftover elements in this fashion: the first element is used to make the root of the tree (since it is roughly halfway between the ends of the list), and the right sub-tree is made by recursively calling itself on the `cdr` of the remaining elements.

b. The asymptotic order of growth is $\Theta(n)$. This is obtained by solving the recurrence relation $T(n) = 2T(n/2) + 1$, since every step, the function calls itself twice on smaller inputs roughly half the size of $n$, and also performs a constant number of steps.

## Exercise 2.65
**Use the results of Exercise 2.63 and Exercise 2.64 to give $\Theta(n)$ implementations of `union-set` and `intersection-set` for sets implemented as (balanced) binary trees.**

In [18]:
; Let's first bring over definitions we need
(define (list->tree elements)
  (car (partial-tree elements (length elements))))
(define (partial-tree elts n)
  (if (= n 0)
    (cons '() elts)
    (let ((left-size (quotient (- n 1) 2)))
      (let ((left-result
              (partial-tree elts left-size)))
        (let ((left-tree (car left-result))
              (non-left-elts (cdr left-result))
              (right-size (- n (+ left-size 1))))
          (let ((this-entry (car non-left-elts))
                (right-result
                  (partial-tree
                    (cdr non-left-elts)
                    right-size)))
            (let ((right-tree (car right-result))
                  (remaining-elts
                    (cdr right-result)))
              (cons (make-tree this-entry
                               left-tree
                               right-tree)
                    remaining-elts))))))))

; Renaming tree->list2 to tree->list
(define (tree->list tree)
  (define (copy-to-list tree result-list)
    (if (null? tree)
        result-list
        (copy-to-list (left-branch tree)
                      (cons (entry tree)
                            (copy-to-list
                              (right-branch tree)
                              result-list)))))
  (copy-to-list tree '()))

(define (left-branch tree) (car tree))
(define (right-branch tree) (caddr tree))
(define (entry tree) (cadr tree))
(define (make-tree entry left-tree right-tree) (list left-tree entry right-tree))

; We will call the previous O(n) definition of union-set that worked on the
; ordered list representation of sets "union-set-list"
(define (union-set-list set1 set2)
  (cond ((null? set1) set2)
        ((null? set2) set1)
        (else (let ((x1 (car set1)) (x2 (car set2)))
                (cond ((= x1 x2) (cons x1 (union-set-list (cdr set1) (cdr set2))))
                      ((< x1 x2) (cons x1 (union-set-list (cdr set1) set2)))
                      ((> x1 x2) (cons x2 (union-set-list set1 (cdr set2)))))))))
; We'll use a similar naming scheme for intersection as well
(define (intersection-set-list set1 set2)
  (if (or (null? set1) (null? set2))
      '()
      (let ((x1 (car set1)) (x2 (car set2)))
        (cond ((= x1 x2)
               (cons x1 (intersection-set-list (cdr set1)
                                          (cdr set2))))
              ((< x1 x2)
               (intersection-set-list (cdr set1) set2))
              ((< x2 x1)
               (intersection-set-list set1 (cdr set2)))))))

; Now the definition of union-set and intersection-set is simple:
(define (union-set set1 set2)
  (define sl1 (tree->list set1))
  (define sl2 (tree->list set2))
  (define slu (union-set-list sl1 sl2))
  (list->tree slu))

(define (intersection-set set1 set2)
  (define sl1 (tree->list set1))
  (define sl2 (tree->list set2))
  (define sli (intersection-set-list sl1 sl2))
  (list->tree sli))

; We can try out these functions on examples
(tree->list (union-set (list->tree '(1 2 5 6)) (list->tree '(1 2 3 4))))

'(1 2 3 4 5 6)

## Exercise 2.66
**Implement the `lookup` procedure for the case where the set of records is structured as a binary tree, ordered by the numerical values of the keys.**

In [19]:
(define (lookup given-key set-of-records)
  (cond ((null? set-of-records) #f)
        ((equal? given-key (key (entry set-of-records))) (key (entry set-of-records)))
        ((< given-key (key (entry set-of-records))) (lookup given-key (left-branch set-of-records)))
        (else (lookup given-key (right-branch set-of-records)))))

To test the above function, we will use a trivial implementation where the whole record is the key:

In [20]:
(define (key x) x)
(define test-tree (list (list '() 1 '()) 2 (list '() 3 '())))

(lookup 3 test-tree)

3

## Exercise 2.67
**Define an encoding tree and a sample message:**
```scheme
(define sample-tree
  (make-code-tree (make-leaf 'A 4)
                  (make-code-tree
                   (make-leaf 'B 2)
                   (make-code-tree
                    (make-leaf 'D 1)
                    (make-leaf 'C 1)))))
(define sample-message `(0 1 1 0 0 1 0 1 0 1 1 1 0))
```
**Use the `decode` procedure to decode the message, and give the result.**

To run the decode procedure, we'll need to bring copy and run the definitions from the text.


In [21]:
(define (make-leaf symbol weight) (list 'leaf symbol weight))
(define (leaf? object) (eq? (car object) 'leaf))
(define (symbol-leaf x) (cadr x))
(define (weight-leaf x) (caddr x))

(define (make-code-tree left right)
  (list left
        right
        (append (symbols left) (symbols right))
        (+ (weight left) (weight right))))

(define (left-branch tree) (car tree))
(define (right-branch tree) (cadr tree))
(define (symbols tree)
  (if (leaf? tree)
      (list (symbol-leaf tree))
      (caddr tree)))

(define (weight tree)
  (if (leaf? tree)
      (weight-leaf tree)
      (cadddr tree)))

(define (decode bits tree)
  (define (decode-1 bits current-branch)
    (if (null? bits)
      '() 
      (let ((next-branch
              (choose-branch (car bits) current-branch)))
        (if (leaf? next-branch)
          (cons (symbol-leaf next-branch)
                (decode-1 (cdr bits) tree))
          (decode-1 (cdr bits) next-branch)))))
  (decode-1 bits tree))

(define (choose-branch bit branch)
  (cond ((= bit 0) (left-branch branch))
        ((= bit 1) (right-branch branch))
        (else (error "bad bit: CHOOSE-BRANCH" bit))))

(define sample-tree
  (make-code-tree (make-leaf 'A 4)
                  (make-code-tree
                   (make-leaf 'B 2)
                   (make-code-tree
                    (make-leaf 'D 1)
                    (make-leaf 'C 1)))))
(define sample-message `(0 1 1 0 0 1 0 1 0 1 1 1 0))

Now we can run `decode` on the sample tree and message.

In [22]:
(decode sample-message sample-tree)

'(A D A B B C A)

## Exercise 2.68
**The `encode` procedure takes as arguments a message and a tree and produces the list of bits that gives the encoded message.**
```scheme
(define (encode message tree)
  (if (null? message)
    '()
    (append (encode-symbol (car message) tree)
            (encode (cdr message) tree))))
```

**`encode-symbol` is a procedure, which you must write, that returns the list of bits that encodes a given symbol according to a given tree. You should design encode-symbol so that it signals an error if the symbol is not in the tree at all. Test your procedure by encoding the result you obtained in Exercise 2.67 with the sample tree and seeing whether it is the same as the original sample message.**


In [23]:
(append '() (list 2))

'(2)

In [24]:
; We'll need a helper function to test if an item is in a list.
; I forgot whether we defined it before, and what we named it, but it is easy enough to define.
(define (list-contains? lst item)
  (if (null? lst) #f (if (eq? item (car lst))
                         #t
                         (list-contains? (cdr lst) item))))

; Defining encode-symbol below:
(define (encode-symbol symbol tree)
  (define (encode-symbol-1 branch partial-code)
    (if (leaf? branch)
        partial-code
        (if (list-contains? (symbols (left-branch branch)) symbol)
            (encode-symbol-1 (left-branch branch) (append partial-code (list 0)))
            (encode-symbol-1 (right-branch branch) (append partial-code (list 1))))))
  (encode-symbol-1 tree '()))

; And copying the definition of encode:
(define (encode message tree)
  (if (null? message)
    '()
    (append (encode-symbol (car message) tree)
            (encode (cdr message) tree))))

; Now running encode-decode on the sample message and tree
(display sample-message)
(newline)
(display (encode (decode sample-message sample-tree) sample-tree))

(0 1 1 0 0 1 0 1 0 1 1 1 0)
(0 1 1 0 0 1 0 1 0 1 1 1 0)

Since the sample message and the sample message after decoding it and again encoding it match, the implementation of `encode-symbol` seems to have worked.

## Exercise 2.69
**The following procedure takes as its argument a list of symbol-frequency pairs
(where no symbol appears in more than one pair) and generates a Huffman
encoding tree according to the Huffman algorithm.**
```scheme
(define (generate-huffman-tree pairs)
  (successive-merge (make-leaf-set pairs)))
```
**`make-leaf-set` is the procedure given above that transforms
the list of pairs into an ordered set of leaves. `successive-merge`
is the procedure you must write, using `make-code-tree` to successively merge
the smallest-weight elements of the set until there is only one element left,
which is the desired Huffman tree. (This procedure is slightly tricky, but 
not really complicated. If you find yourself designing a complex procedure,
then you are almost certainly doing something wrong. You can take
significant advantage of the fact that we are using an ordered set representation.)**

In [25]:
; First we need to bring over the definition of adjoin-set
(define (adjoin-set x set)
  (cond ((null? set) (list x))
        ((< (weight x) (weight (car set))) (cons x set))
        (else (cons (car set)
                    (adjoin-set x (cdr set))))))
; and make-leaf-set
(define (make-leaf-set pairs) 
 (if (null? pairs) 
  '() 
  (let ((pair (car pairs))) 
   (adjoin-set (make-leaf (car pair) 
                (cadr pair)) 
    (make-leaf-set (cdr pairs))))))

; Now, here is the definition of successive-merge
(define (successive-merge nodes)
  (if (= (length nodes) 1)
      (car nodes)
      (let ((left (car nodes))
            (right (cadr nodes))
            (rest (cddr nodes)))
        (successive-merge (adjoin-set (make-code-tree left right) rest)))))

; Now, the generate-huffman-tree definition should work too
(define (generate-huffman-tree pairs)
  (successive-merge (make-leaf-set pairs)))

## Exercise 2.70
**The following eight-symbol alphabet with
associated relative frequencies was designed to efficiently
encode the lyrics of 1950s rock songs. (Note that the “symbols” of an “alphabet”
need not be individual letters.)**
```
A 2     GET 2   SHA 3   WAH 1
BOOM 1  JOB 2   NA 16   YIP 9
```
**Use generate-huffman-tree (Exercise 2.69) to generate a
corresponding Huffman tree, and use encode (Exercise 2.68)
to encode the following message:**

```
Get a job
Sha na na na na na na na na
Get a job
Sha na na na na na na na na
Wah yip yip yip yip yip yip yip yip yip
Sha boom
```

**How many bits are required for the encoding? What is the
smallest number of bits that would be needed to encode this
song if we used a fixed-length code for the eight-symbol
alphabet?**



In [26]:
; Instead of doing string processing, I did some tedious manual text editing and converted both the
; symbols of the alphabet with their frequencies and the song to the data structures we need.
(define alphabet (list (list 'A 2) (list 'GET 2) (list 'SHA 3) (list 'WAH 1) (list 'BOOM 1) (list 'JOB 2) (list 'NA 16) (list 'YIP 9)))
(define song (list 'GET 'A 'JOB 'SHA 'NA 'NA 'NA 'NA 'NA 'NA 'NA 'NA 'GET 'A 'JOB 'SHA 'NA 'NA 'NA 'NA 'NA 'NA 'NA 'NA 'WAH 'YIP 'YIP 'YIP 'YIP 'YIP 'YIP 'YIP 'YIP 'YIP 'SHA 'BOOM))

; Now we can generate the tree
(define tree (generate-huffman-tree alphabet))
; and encode the song using the tree
(define code (encode song tree))

; Decoding the encoded song using the same tree gives us the song back.
(decode code tree)

'(GET A JOB SHA NA NA NA NA NA NA NA NA GET A JOB SHA NA NA NA NA NA NA NA NA WAH YIP YIP YIP YIP YIP YIP YIP YIP YIP SHA BOOM)

To answer the question, we can take the length of the encoded version:

In [27]:
(length code)

84

A fixed-length code will need 3 bits to encode each symbol in the alphabet, since $2^3 = 8$. Since the song is made of 36 symbols, with a fixed-length code, we will need $36 * 3 = 108$ bits to encode the song.

## Exercise 2.71
**Suppose we have a Huffman tree for an alphabet of $n$ symbols,
and that the relative frequencies of the symbols are $1, 2, 4,..., 2^{n-1}$.
Sketch the tree for $n=5$; for $n=10$. In such a tree (for general $n$) how many
bits are required to encode the most frequent symbol? The least frequent 
symbol?**

Here is a 'sketch' of the tree for $n=5$:

![Tree for 5 symbols](aux/2.71/tree.png)

And here for $n=10$:

![Tree for 10 symbols](aux/2.71/tree-10.png)

The most frequent symbol will always appear as a child of the root node in such a tree, so one bit is sufficient to encode the most frequent symbol. For the least frequent symbol, $n-1$ bits will be required.

## Exercise 2.72
**Consider the encoding procedure that you designed in Exercise 2.68. What is the order of growth in
the number of steps needed to encode a symbol? Be sure to include the number of steps needed to search
the symbol list at each node encountered. To answer this question in general is difficult.
Consider the special case where the relative frequencies of the $n$ symbols are as described in Exercise 2.71,
and give the order of growth (as a function of $n$) of the number of steps needed to encode the most frequent
and least frequent symbols in the alphabet.** 


I think it will be easier to start with the worst case here, i.e. the types of trees constructed in Exercise 2.71.

Tackling one problem at a time, let's say for now that $s(n)$ is the number of steps required to search for an item in a set with $n$ elements. For the most frequent element, we will need two searches: once on the node at the root of the subtree with all the other elements, and once on the leaf with the most frequent element. This is $s(n-1)$, so basically $O(s(n-1))$ steps are required to encode the most frequent element. (It would be $s(1)$ if we checked the other node first, but I guess we need to stick to the worst case here.)

For the least frequent element, we will need to repeat this step down all the levels, so the number of steps will be $s(n-1) + s(n-2) + s(n-3) + ... + s(1)$, or $\sum_{i=1}^{n-1}s(n-i)$. The value of $s(n)$ will depend on the specific implementation of the set data structure, but let's use $s(n)=\log{n}$, which will be the case for sets implemented as binary search trees. So, the number of steps is $s(n-1) + s(n-2) + ... + s(1) = \log{(n-1)} + \log{(n-2)} + ... + \log{1} = \log{((n-1)(n-2)...(2)(1))}=\log{(n-1)!}$

On the other end of the spectrum, if we have a very balanced tree, then instead of going down the tree for $(n-1)$ steps, we will only need to go down about $\log(n)$ steps. Also, instead of the number of items we need to search on decreasing by one, they will be halved at every step. So the number of steps needed is $\log{\frac{n}{2}} + \log{\frac{n}{4}} + \log{\frac{n}{8}} + ... = \log{(\frac{n^{\log{n}}}{{2^{(\log{n})(1+\log{n})/2}}})} = \log{(\frac{n^{\log{n}}}{{n^{(1+\log{n})/2}}})}=\log{n^{\log{n} - (1+\log{n})/2}}=\log{n^{\frac{\log{n}-1}{2}}}=\frac{(\log{n}-1)(\log{n})}{2}$
